# Exercise: Scaling of domain wall thickness with wire width, using mumax3

## Micromagnetic material parameters

With respect to the question _"How much can we miniaturise domain wall devices?"_, here we want to compare the shape-dependent domain wall width for common materials permalloy (Py, Fe$_{0.2}$Ni$_{0.8}$), which is a soft magnet with $K_1\approx0$, and CoPt, which is a material with a strong perpendicular magnetic anisotropy.

These materials have the following material parameters (at room temperature):

- Py: $M_\text{sat}$ = 840 kA/m, $A_\text{ex}$ = 10 pJ/m, $K_1\approx 0$, $T_C$ = 843 K
- CoPt: $M_\text{sat}$ = 810 kA/m, $A_\text{ex}$ = 10 pJ/m, $K_1$ = 4900 kJ/m$^3$, $T_C$ = 840 K

__Group Assignment for collaborative simulations__

- material: odd birthdays - CoPt, even birthdays - Py
- width: e.g. try different multiples of your favourite prime number

__Task:__

- Calculate the exchange length $l_\text{ex}=\sqrt{\frac{A_\text{ex}}{\mu_0 M_\text{sat}^2}}$ for each material!

__To discuss__:

- What does $l_\text{ex}$ represent, and how is it relevant for micromagnetic simulations?
- Which material do you expect to have thinner domain walls, and why?

In [ ]:
""" TASK: Calculate the exchange length.  """

## Simulate a domain wall with mumax3

Mumax is a GPU-accelerated finite-difference micromagnetic simulator, i.e., sample geometries are approximated by a regular grid of cuboid volumes.

- Webpage of mumax3: http://mumax.github.io/index.html

Since I do not expect that your laptops have an NVIDIA graphics card with CUDA, we can use a mumax google collaboratory online:

- Browse to https://colab.research.google.com/github/JeroenMulkers/mumax3-tutorial/blob/master/mumax3.ipynb
- Run the installation of mumax, as per instructions in the colaboratory notebook. 
- This may take a while - during that time you can have a look at the mumax simulation script with ending `*.mx3`. 
    - Do you have questions? Are things unclear?
    - Put in missing values, according to the situation you want to simulate.
- To run the simulation, copy the instruction file `wall_width.mx3` into the online folder, and run a cell with the instruction `!mumax3 wall_width.mx3`. Alternatively, create a new file with the correct file ending and copy the instructions below.

### example mumax simulation file

## import OVF file and plot magnetisation profile

I often use `pandas` dataframes for data analysis, as it is easier to keep track on what is what for arrays with many columns.

We use these functions below to plot the averaged magnetisation profile from the simulations and extract the domain wall width.

- Copy the next two notebook cells into your collaboratory notebook to have access to the data loader and plotting helper functions.

Inspect data:

- Load data in pandas dataframe with columns ['x', 'y', 'z', 'mx', 'my', 'mz'].
- Plot magnetisation profile along $x$. 
    - Handy `pandas` function to get $m_x(x)$: `data.groupby(by='x').mean()['mx']`.
- Is shape as expected for the initial domain wall type? 
- Do you observe deviations, and if yes, where could they originate from? If in doubt, you can check the snapshots of the simulation.

Extract the domain wall width. For simplicity, we will define it as the 10%-90% interval.

- Is the simulation size large enough to extract a valid 10%-90% domain wall width?
- Extract wall width.
    - Write a small function to get the value `wall_width`.
- Is there a difference in Neel and Bloch domain wall width?

Wall width vs. wire width:

- Repeat simualtions for several wire widths and the two materials. To make it collaborative:
    - material: odd birthdays - CoPt, even birthdays - Py
    - width: e.g. try different multiples of your favourite prime number
- Keep in mind for each choice of wire width that the following conditions need to be fulfilled:
    - wire width = #cells * cell size
    - #cells = $2^N$
    - cell size < $l_{ex}$
    
Discuss:
   
- Let's plot collaboratively the relationship wire width vs. wall width. What do we observe?
- Compare with the domain wall width parameter $\delta_w = \pi \sqrt{\frac{A_\text{ex}}{K_1}}$!
- What materials would you use to build devices?

In [76]:
""" This cell defines helper functions """

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def read_ovf_metadata(filename):
    """ Reads OVF metadata and returns a dictionary """
    metadata = {}
    with open(filename, "r") as f:
        txt = f.readlines()
        for line in txt:
            if line.startswith ('# '):
                word = line[2:].split(': ')
                if len(word) == 2:
                    metadata[word[0]] = word[1][:-1]
    return metadata


def read_mumax_to_pandas( filename_m, filename_geo='', filename_Msat='', filename_B='', centre_xy=False, cell_offset=True ):
    """ 
    Read mumax ovf files (stored with specification 'OutputFormat = OVF2_TEXT')
    to a pandas dataframe for involved data plotting.
    
    :Parameters: 
    *filename_m*
        Magnetisation vectors (normalised to 1)
    *filename_B* 
        Demagnetisation field.
    *filename_Msat* 
        Local saturation magnetisation.
    *filename_geo* 
        Specifies the regions (and thus geometry) of the simulation.
    *centre_xy* Bool, default False
        Centres lateral coordinates.
    *cell_offset* Bool, default True
        Add offsets of the mumax cells (i.e. half the cell size in each direction)
    
    :Returns:
        Pandas dataframe with the follwing columns:
            x, y, z, mx, my, mz
            optional columns, if specified:
            geo, Msat, B
        shape of the grid (can be used to re-shape for tri-mesh plotting etc...)
    """
    
    import numpy as np
    import pandas as pd
    
    # load metadata
    metadata = read_ovf_metadata(filename_m)
    grid_shape = [int(metadata[f'{direction}nodes']) for direction in ['x', 'y', 'z'] ]
    
    # load magnetisation data (data_m)
    data = pd.read_csv( filename_m, comment='#', delimiter='\s+', names=['mx', 'my', 'mz'], dtype='float')
    # one can calculate other properties from mx, my, mz, such as the in-plane magnetisation angle etc
    
    # set position coordinates 
    # 'xstepsize': '3.125e-09', vs. 'xbase': '1.5625e-09',
    x_list, y_list, z_list =\
        [ np.arange( int(metadata[f'{direction}nodes']) ) * float(metadata[f'{direction}stepsize']) \
        for direction in ['x', 'y', 'z'] ]
    X_grid, Y_grid, Z_grid = np.meshgrid( x_list, y_list, z_list, indexing='ij' )
    if centre_xy == True:
        X_grid = X_grid - 0.5 * np.max(X_grid)
        Y_grid = Y_grid - 0.5 * np.max(Y_grid)
    data['x'] = np.transpose(X_grid).ravel() + cell_offset * float(metadata['xbase'])
    data['y'] = np.transpose(Y_grid).ravel() + cell_offset * float(metadata['ybase'])
    data['z'] = np.transpose(Z_grid).ravel() + cell_offset * float(metadata['zbase'])
    
    # load geometry specifications
    if len(filename_geo) > 0:
        data[f'geo'] = np.loadtxt( filename_geo, delimiter='\n', comments='# ' ).astype(int)
    
    # load saturation magnetisation
    # alternatively, these values can be set via the geometry specifiers
    if len(filename_Msat) > 0:
        data['Msat'] = np.loadtxt( filename_Msat, delimiter='\n', comments='# ' ).astype(float)
        
    # load demagnetisation field
    if len(filename_B) > 0:
        data_B = pd.read_csv( filename_B, comment='#', delimiter='\s+', names=['Bx', 'By', 'Bz'], dtype='float')
        for direction in ['x', 'y', 'z']:
            data[f'B{direction}'] = data_B[f'B{direction}']

    return data, grid_shape


def plot_magnetisation_profile( filename, figure_title, m_col_switch='mx' ):
    """  Plot magnetisation profiles, get wall width for switched m_i """

    # load data
    data, grid_shape = read_mumax_to_pandas(filename, centre_xy=True )

    # wall width == mx goes from +0.8 to -0.8 (10% to 90% of signal)
    x_list = np.unique(data['x'])
    mx_profile = np.array(data.groupby(by='x').mean()[m_col_switch])
    x_10p = 0 """ TODO: determine 10% level """
    x_90p = 0 """ TODO: determine 90% level """
    wall_width = np.abs( x_90p - x_10p ) * 1e9 # in nm

    # plot magnetisation profile
    fig, ax = plt.subplots()
    ax.set_title(figure_title)
    for i, m_col in enumerate(['mx', 'my', 'mz']):
        ax.plot(
              np.unique(data['x'])
            , data.groupby(by='x').mean()[m_col]
            , label=m_col
            , linestyle = ['-', '--', ':'][i]
            )

    # helper lines
    for level in [-1, -0.8, 0, 0.8, +1]:
        ax.axhline(level, alpha=0.5, color='black', linewidth=0.5, zorder=-1)
    ax.axvline(0, alpha=0.5, color='black', linewidth=0.5, zorder=-1)
    ax.legend(title=f'wall width = {wall_width:.1f} nm', loc='lower left')

    return

In [ ]:
# running mumax simulations in collaboratory
#!mumax3 wall_width.mx3

# plotting results
wire_width, m_col_switch = 25, 'mx'
plot_magnetisation_profile( f'./wall_width.out/{wire_width}nm_Bloch.ovf', f'Bloch wall, {wire_width} nm', m_col_switch=m_col_switch )
plot_magnetisation_profile( f'./wall_width.out/{wire_width}nm_Neel.ovf', f'Néel wall, {wire_width} nm', m_col_switch=m_col_switch )

## For exploration: Domain wall creep in magnetic Galton board

If you are interested, I also attach another mumax script for the Galton board to play with. Please note that I do not have yet the answers to this either ;-)

Consider the following questions:

- How does the shape of the device help to make the position of domain wall nucleation deterministic?
- Find the reversal field at which the domain wall creep starts and see whether you get stochastic choice of the wall going left or right.
- In the paper you'll find that the device shape was finely tuned, with curved parts at the top of each row - can you guess why this seemed useful and do you see the difference in the simulated behaviour?